<a href="https://colab.research.google.com/github/ma850419/FlexibleNet/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from osgeo import gdal
filepath = r'/content/PRS_L1_STD_OFFL_20221217082013_20221217082018_0001_HCO_VNIR.tif'
# https://stackoverflow.com/questions/43684072/how-to-import-multiple-bands-from-an-image-into-numpy
# Load one GeoTIFF image using GDAL
dataset = gdal.Open(filepath)
#nodata_value = dataset.GetRasterBand(1).GetNoDataValue()
projInfo = dataset.GetProjection()
trans = dataset.GetGeoTransform()
print(projInfo,trans)
#image = np.zeros(( dataset.RasterXSize,dataset.RasterYSize,dataset.RasterCount))
image = dataset.ReadAsArray()
image2 = np.transpose(image,(1,2,0))
  # <class 'numpy.ndarray'>
print(image2.shape) 
print(image.dtype)  
print()

GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]] (35.49288558959961, 0.0003509521484375, 0.0, 33.71284866333008, 0.0, -0.0002593994140625)
(1231, 1245, 63)
float32



In [16]:
#rast_arr= np.transpose(image2,(2,0,1))
rast_arr=image
print(rast_arr.shape)

(63, 1231, 1245)


In [2]:
!pip install minisom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for minisom: filename=MiniSom-2.3.1-py3-none-any.whl size=10609 sha256=2d3afb4169c41bd03e1fb383d15faded3a5b1446033435384851bc606dca41f6
  Stored in directory: /root/.cache/pip/wheels/b4/aa/7a/d9a88098f7877aa95dd4f227fe614f75773654baa39b47bba6
Successfully built minisom


In [3]:
import statistics
from minisom import MiniSom
mine=np.zeros((1231*1245,63))
pixels2=np.zeros((1231*1245,63))
pixels3=np.zeros((1231*1245,63))
som_clustered=np.zeros((1231*1245,63))
som = MiniSom(4, 4, 63, sigma=0.5,learning_rate=0.1, neighborhood_function='gaussian')
#mine=zeros((512*512,4))
#pixels2=zeros((600,512*512,4))
#som_clustered=zeros((600,512*512,1))
#for i in range(600):
pixels2= image2.reshape(1231*1245,63)
pixels3 = pixels2 != -999
pixels2 = abs(pixels2)*pixels3/300.0
#print( pixels2.min())
som.random_weights_init(pixels2)
starting_weights = som.get_weights().copy()  # saving the starting weights
som.train(pixels2, 10000, random_order=False, verbose=True)
print('quantization...')
qnt = som.quantization(pixels2)  # quantize each pixels of the image
    #print(qnt.shape)
clustered = np.zeros((1231*1245,63))
clustered1 = np.zeros((1231*1245,1))
for j, q in enumerate(qnt):  # place the quantized values into a new image
  sn = np.unravel_index(j,shape=((1231*1245)))
    #print(sn)
    #print(me)
   
  #print(q,(mean(q)*6).astype(np.uint8))
  #print(me)
  clustered[sn] = q[:] #(max(q)*6).astype(np.uint8) 
  #print(q[:]*255,round(q[:].max()*255))
  #print(me)
  #print((round(q[:].max()*16*300.0)))
  #print(me)
  clustered1[sn] = (round(q[:].max()*255))#.astype(np.uint8) #statistics.mean(q)
 # print(clustered1[sn])
        #print(clustered.shape)
#mine= np.append(mine,clustered,axis=0)#,axis=0)
   # print(clustered1)
#som_clustered[i] =  clustered1
    #print(som_clustered[i])
    #print(me)
#print(mine.shape)

 [ 10000 / 10000 ] 100% - 0:00:00 left 
 quantization error: 0.09250485365786938
quantization...


In [ ]:
import cv2
import matplotlib.pyplot as plt
#print(clustered.shape, clustered1.shape)
pixels2= pixels2.reshape((1231,1245,63))
clustered2 = np.zeros((1231,1245))
clustered3 = np.zeros((1231,1245,3))
clustered3[:,:,0]=pixels2[:,:,60]
clustered3[:,:,1]=pixels2[:,:,35]
clustered3[:,:,2]=pixels2[:,:,20]
clustered3 = cv2.pow(clustered3,0.6)
# Convert the image from BGR to HSV color space
clustered2 = clustered1.reshape(1231,1245)
f, axarr = plt.subplots(2,2)
plt.figure(figsize = (20,4))
axarr[0,0].imshow(clustered2)
#plt.show()
axarr[0,1].imshow(clustered3)
#plt.show()
clustered3[:,:,0]=pixels2[:,:,50]
clustered3[:,:,1]=pixels2[:,:,30]
clustered3[:,:,2]=pixels2[:,:,15]
clustered3 = cv2.pow(clustered3,0.7)
axarr[1,0].imshow(clustered3)
#plt.show()
clustered3[:,:,0]=pixels2[:,:,30]
clustered3[:,:,1]=pixels2[:,:,20]
clustered3[:,:,2]=pixels2[:,:,10]
axarr[1,1].imshow(clustered3)
plt.show()

In [ ]:
!ls -l

In [23]:
#from osgeo import osr
output_file = "out7.tif"
#clustered2 = np.zeros((1231,1245))
#clustered2 = clustered1.reshape(1231,1245)
# Create gtif
#rast_arr= np.zeros((image.shape))
#rast_arr= np.copy((image))
#print(rast_arr)
#driver = gdal.GetDriverByName("GTiff")
#x_size = dataset.RasterXSize # Raster xsize
#y_size = dataset.RasterYSize # Raster ysize
nband = dataset.RasterCount # number of bands
NaN_rast=-999
#dst_ds = driver.Create(output_file, x_size, y_size,nband, gdal.GDT_Float32 )
# top left x, w-e pixel resolution, rotation, top left y, rotation, n-s pixel resolution
#dst_ds.SetGeoTransform( [ -180, 0.025, 0, 90, 0, -0.025 ] )
#print(clustered2[900:1000,900:1000], clustered2.dtype)
#print(me)
# set the reference info 
band = 0
if type(image) == tuple:
  print('0')
  rast_arr = np.array(image[band])
if str(type(dataset)) == "<class 'osgeo.gdal.Dataset'>":
  print('2')
  geo_transform = dataset.GetGeoTransform()
  x_size = dataset.RasterXSize  # Raster xsize
  y_size = dataset.RasterYSize  # Raster ysize
  #print(x_size,y_size)
  srs = dataset.GetProjectionRef()  # Projection
elif str(type(dataset)) == "<class 'affine.Affine'>":
  print('3')
  geo_transform = (dataset[2], dataset[0], dataset[1], dataset[5], dataset[3], dataset[4])
  rast_arr = image[band,:,:]
  x_size = int(rast_arr.shape[1])
  y_size = int(rast_arr.shape[0])
driver = gdal.GetDriverByName("GTiff")
dataset_out = driver.Create(output_file, x_size, y_size, nband, gdal.GDT_Float32)
    #end auxiliar
for band in range(1,nband):
  if type(image) == tuple:
    print('Iam ')
    rast_arr = np.array(image[band])
  if str(type(dataset)) == "<class 'osgeo.gdal.Dataset'>":
    print('Iam here')
    geo_transform = dataset.GetGeoTransform()
    x_size = dataset.RasterXSize  # Raster xsize
    y_size = dataset.RasterYSize  # Raster ysize
    #print(x_size,y_size)
    srs = dataset.GetProjectionRef()  # Projection
  elif str(type(dataset)) == "<class 'affine.Affine'>":
    print('Iam here here')
    geo_transform = (dataset[2], dataset[0], dataset[1], dataset[5], dataset[3], dataset[4])
    rast_arr = image[band,:,:]
    x_size = int(rast_arr.shape[1])
    y_size = int(rast_arr.shape[0])
        #PROCESS RASTERIO NUMPY
  else:
    print('Iam here here here')
    geo_transform = (dataset[1][2], dataset[1][0], dataset[1][1], dataset[1][5], dataset[1][3], dataset[1][4])
    rast_arr = np.array(dataset[0])
    x_size = int(rast_arr.shape[2])
    y_size = int(rast_arr.shape[1])
  rast_arr[rast_arr == NaN_rast] = np.NaN
  dataset_out.SetGeoTransform(geo_transform)
  dataset_out.SetProjection(srs)
  print(band)
  dataset_out.GetRasterBand(band).WriteArray(rast_arr[band,:,:])#.astype(np.float32))
#for band in range(1,nband):
  #dst_ds.SetGeoTransform(trans)
#dst_ds.ImportFromWkt(projInfo)
  #dst_ds.SetProjection( projInfo )
  #dst_ds.GetRasterBand(band).WriteArray(image)
#dst_ds.FlushCache()
#dst_ds = None


2
Iam here
1
Iam here
2
Iam here
3
Iam here
4
Iam here
5
Iam here
6
Iam here
7
Iam here
8
Iam here
9
Iam here
10
Iam here
11
Iam here
12
Iam here
13
Iam here
14
Iam here
15
Iam here
16
Iam here
17
Iam here
18
Iam here
19
Iam here
20
Iam here
21
Iam here
22
Iam here
23
Iam here
24
Iam here
25
Iam here
26
Iam here
27
Iam here
28
Iam here
29
Iam here
30
Iam here
31
Iam here
32
Iam here
33
Iam here
34
Iam here
35
Iam here
36
Iam here
37
Iam here
38
Iam here
39
Iam here
40
Iam here
41
Iam here
42
Iam here
43
Iam here
44
Iam here
45
Iam here
46
Iam here
47
Iam here
48
Iam here
49
Iam here
50
Iam here
51
Iam here
52
Iam here
53
Iam here
54
Iam here
55
Iam here
56
Iam here
57
Iam here
58
Iam here
59
Iam here
60
Iam here
61
Iam here
62


In [ ]:
def unique(list1):
 
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    #for x in unique_list:
        #print(x)
    return unique_list

In [ ]:
list1=[]
list1= np.append(list1,clustered)
#list1=clustered1.tolist()
l= np.zeros(60)
u=unique(list1)
print(u)


In [ ]:
a=plt.hist(clustered)
print(a)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
#from keras.utils import to_categorical
from keras.preprocessing import image
#import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import  confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
#from keras.utils import to_categorical
from tqdm import tqdm
#import pydot
#import pydot_ng as pydot
from keras.utils import plot_model
from keras.layers import Input, Dense, BatchNormalization

In [ ]:
#creating subimage of size 32X32
height=32
width = 32
subim = np.zeros((38,32,32,63))
imgwidth, imgheight, bands= image2[15:1231,29:1245,:].shape
image3 = np.zeros((imgwidth, imgheight, bands))
image3 = image2[15:1231,29:1245,:]
#print(imgwidth, imgheight, bands)
for k in range(0,38):
  for i in range(0,imgheight,height):
    for j in range(0,imgwidth,width):
      #box = (j, i, j+width, i+height)
      subim[k]= image3[j:j+width,i:i+height,:]

In [ ]:
import csv
from os import listdir
import numpy as np
import matplotlib.image as img
import pandas as pd
import os
from os.path import exists
from pathlib import Path

#with open('C:/Users/ma850/OneDrive/Documents/banana_spectral_signatures/banana_healthy_1a.dat', 'r') as dat_file:
 #   with open('C:/Users/ma850/OneDrive/Documents/banana_spectral_signatures/banana_healthy_1a.csv', 'w') as csv_file:
csv_dir =  r'/content/'
onlyfilenames = [f for f in listdir(csv_dir) if os.path.isfile(csv_dir+f)]
for file in onlyfilenames:
    print(file)
    filename, extension = os.path.splitext(file)
    if extension == '.csv':
        df = pd.read_csv(csv_dir+file,header =0)#, chunksize=1000)
        print(df.shape)
specsig = (df.iloc[:,:].values).astype('float32')
print(specsig.shape)

In [ ]:
#filtering A spectril signature

import numpy
print(specsig[:,0])
specsig[:,0:1]
import matplotlib.pyplot as plt
#plt.plot(specsig[:,0],specsig[:,1])
specsig1 = numpy.zeros((2151,30))
specsig2 = numpy.zeros((2151,30))
leng=len(specsig)
#print(leng)
specsig1[:,0] = specsig[:,0]
for i in range(leng):
    for j in range(1,30,1):
        if(specsig[i,j] >=0 and specsig[i,j] <= 1.0):
            specsig1[i,j]=specsig[i,j]
        else:
            specsig1[i,j]= 0
#print(specsig1[:,0:1])       
specsig2[:,0] = specsig[:,0]
specsig2[0:991,1:30]=specsig[0:991,1:30]
specsig2[991:1096,1:30]=specsig[991:1096,1:30]
specsig2[1096:1441,1:30]= specsig[1096:1441,1:30]
specsig2[1441:1606,1:30]=0
specsig2[1606:2151,1:30]= specsig[1606:2151,1:30]
print(specsig2.shape)
plt.subplot(1, 2 , 1) # row 1, col 2 index 1
#plt.plot(specsig2[0:991,0],specsig2[0:991,2],'b.')
#plt.plot(specsig2[1096:1441,0],specsig2[1096:1441,2],'b.')
#plt.plot(specsig2[1606:2100,0],specsig2[1606:2100,2],'b.')
plt.plot(specsig2[:,0],specsig2[:,1:],'b.')
plt.subplot(2, 2, 2) 
plt.plot(specsig[:,0],specsig[:,1:],'r^')
plt.show

In [ ]:
# reading FWHM file for prisma
csv_dir = "/content/"
file='wvfwmh_VNIR.csv'
df = pd.read_csv(csv_dir+file,header =0)
print(df.shape)
fwhm= (df.iloc[:,:].values).astype('float32')

In [ ]:
#resmapling spectral signatures to Prisma VNIR using FWHM
specsig3 = np.zeros((63,31))
specsig3[:,0]=fwhm[:,2]
specsig3[:,1]=fwhm[:,3]
for j in range(1,30):
   # n=0
    for i in range(0,63):
        f= round(fwhm[i,2])
        k= round(f+fwhm[i,3])
        s=0
        l=0
        for m in range (0,2151):
           
            if specsig2[m,0] >= f and specsig2[m,0]<=k:
                s= specsig2[m,j]+ s
                l=l+1
        av= s/l
        specsig3[i,j+1]= av
 